In [1]:
from pathlib import Path
import numpy as np
from enum import Enum
from typing import Tuple
from dataclasses import dataclass

In [2]:
def parse_map(lines):
    width = max(len(l) for l in lines)
    M = [list(l + ' ' * (width-len(l))) for l in lines]
    return np.array(M)
        
def parse_inst(insts):
    cur = ''
    out = []
    for c in insts:
        if c in ['L','R']:
            if len(cur) > 0:
                out.extend([int(cur),c])
            else:
                out.append(c)
            cur = ''
        else:
            cur = cur + c
    if len(cur) > 0:
        out.append(int(cur))
    return out




In [3]:
def parse_cube(lines):
    sz = int(len(lines[0])/3)
    M = np.zeros((6,sz,sz), dtype='U1')
    k = 0
    N = {}
    O = {}
    for i in range(0,len(lines), sz):        
        for j in range(0, len(lines[i]), sz):
            if lines[i][j] != ' ':
                M[k] = np.array([[c for c in l[j:j+sz]] for l in lines[i:i+sz]], dtype='U1')
                O[k] = (i,j)
                N[(i // sz, j//sz)] = k
                k +=1

                
                

    assert k == 6
    return M, N, O
        

In [4]:
class Facing(Enum):
    right = 0
    down = 1
    left = 2
    up = 3

    def turnR(self):
        val = (self.value +1) % 4
        return Facing(val)

    def turnL(self):
        val = (self.value -1) % 4
        return Facing(val)

    def turn(self, d):
        if d == 'L':
            return self.turnL()
        elif d == 'R':
            return self.turnR()
        else:
            raise Exception(f'unknown direction {d=}')

In [5]:
def read(prefix='data', part=1):
    M, I = Path(f'{prefix}/22.txt').read_text().rstrip().split('\n\n')
    if part == 1:
        M = parse_map(M.rstrip().split('\n'))
    else:
        M = parse_cube(M.rstrip().split('\n'))
    return M, parse_inst(I)


In [8]:
class Map:
    def __init__(self, M):
        self._M = M
        self.x = list(M[0]).index('.')
        self.y = 0
        self.facing = Facing.right
        self.__set_boundary()
    
    def __set_boundary(self):
        self.edgeL, self.edgeR = [], []
        for i in range(self._M.shape[0]):
            onmap = np.where(self._M[i] != ' ')[0]
            self.edgeL.append(onmap[0])
            self.edgeR.append(onmap[-1])

        self.edgeT, self.edgeB = [], []
        for i in range(self._M.shape[1]):
            onmap = np.where(self._M[:,i] != ' ')[0]
            self.edgeT.append(onmap[0])
            self.edgeB.append(onmap[-1])


    def move(self, inst):
        if inst in ['L', 'R']:
            self.facing = self.facing.turn(inst)
        else:
            inst = int(inst)
            self.forward(inst)

    def forward(self, num_steps : int):
        assert num_steps >= 0
        for _ in range(num_steps):
            moved = self.step()
            if not moved:
                return

    def step(self):
        match self.facing:
            case Facing.up:
                if self.y == self.edgeT[self.x]:
                    x_n, y_n = self.x, self.edgeB[self.x]
                else:
                    x_n, y_n = self.x, self.y - 1
            case Facing.down:
                if self.y == self.edgeB[self.x]:
                    x_n, y_n = self.x ,self.edgeT[self.x]
                else:
                    x_n, y_n = self.x, self.y + 1
            case Facing.left:
                if self.x == self.edgeL[self.y]:
                    x_n, y_n = self.edgeR[self.y], self.y
                else:
                    x_n, y_n = self.x - 1, self.y
            case Facing.right:
                if self.x == self.edgeR[self.y]:
                    x_n, y_n = self.edgeL[self.y], self.y
                else:
                    x_n, y_n = self.x + 1, self.y
            
        can_move = (self._M[y_n, x_n] == '.')
        if can_move:
            self.x, self.y = x_n, y_n
        return can_move


    def disp(self):
        print(*[''.join(r) for r in self._M], sep='\n')

    def password(self):
        return ((self.y+1) * 1000)  + ((self.x+1) * 4) + self.facing.value

In [9]:
def part1(prefix='data'):
    M, I = read(prefix)
    map = Map(M)
    for inst in I:
        map.move(inst)
    return map.password()

In [10]:
part1('test')

6032

In [11]:
part1()

66292

In [12]:
class Point():
    def __init__(self, x,y,z):
        self._t = (x,y,z)

    def __add__(self, other):
        t = tuple(s+o for (s,o) in zip(self._t, other._t))
        return Point(*t)
    
    def __sub__(self, other):
        t = tuple(s-o for (s,o) in zip(self._t, other._t))
        return Point(*t)

    def __abs__(self):
        t = tuple(abs(s) for s in self._t)
        return Point(*t)

    def __repr__(self):
        return repr(self._t)


@dataclass
class Side:
    topleft : Point
    topright : Point
    bottomleft : Point
    bottomright : Point

    @property
    def coord_x(self):
        return abs(self.topright - self.topleft)

    @property
    def coord_y(self):
        return abs(self.topleft - self.bottomleft)

    @property
    def coord_other(self):
        return Point(1,1,1) - (self.coord_x + self.coord_y)

    def left(self):
        topright = self.topleft
        bottomright = self.bottomleft
        topleft = abs(self.topleft - self.coord_other)
        bottomleft = abs(self.bottomleft - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)
        
    def right(self):
        topleft = self.topright
        bottomleft = self.bottomright
        topright = abs(self.topright - self.coord_other)
        bottomright = abs(self.bottomright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)

    def up(self):
        bottomleft = self.topleft
        bottomright = self.topright
        topleft = abs(self.topleft - self.coord_other)
        topright = abs(self.topright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)

    def down(self):
        topleft = self.bottomleft
        topright = self.bottomright
        bottomleft = abs(self.bottomleft - self.coord_other)
        bottomright = abs(self.bottomright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)

    @property
    def topedge(self):
        return (self.topleft, self.topright)

    @property
    def leftedge(self):
        return (self.topleft, self.bottomleft)

    @property
    def rightedge(self):
        return (self.topright, self.bottomright)

    @property
    def bottomedge(self):
        return (self.bottomleft, self.bottomright)


In [13]:
from collections import Counter
def find_corners(N):
    found = [(k, v) for (k,v) in N.items() if v == 0]
    origin = Side(Point(0,0,0), Point(1,0,0), Point(0,1,0), Point(1,1,0))
    corners = {0 : origin}
    while len(found) > 0:
        (i,j), k = found[0]
        found = found[1:]
        
        cur = corners[k]
        nbs = [(i-1,j), (i+1,j), (i,j+1), (i,j-1)]
        sides = [cur.up(), cur.down(), cur.right(), cur.left()]
        for (nb, side) in zip(nbs, sides):
            if nb in N:
                if not N[nb] in corners:
                    corners[N[nb]] = side
                    found.append((nb, N[nb]))

    assert set(corners) == set(range(6))

    L = []
    for side in corners.values():
        L.extend([side.topleft._t, side.topright._t, side.bottomleft._t, side.bottomright._t])
    C = Counter(L)
    assert max(C.values()) == 3 and min(C.values()) == 3 

    return corners

In [14]:
from collections import defaultdict
def find_edges(corners):
    edges = defaultdict(list)
    for k, side in corners.items():
        TLBR = [(side.topedge, 'T'), (side.leftedge, 'L'), (side.bottomedge, 'B'), (side.rightedge, 'R')]
        for ((start, end), name) in TLBR:
            edges[(start._t, end._t)].append((k, name,1))
            edges[(end._t, start._t)].append((k, name,-1))
    return dict(edges)

In [15]:
def make_transitions(edges):

    f_src = {'L' : Facing.left, 'R' : Facing.right, 'T' : Facing.up, 'B' : Facing.down}
    f_dst = {'L' : Facing.right, 'R' : Facing.left, 'T' : Facing.down, 'B' : Facing.up}
    T = {}
    for ((s_k,s_e, s_r),(t_k,t_e, t_r)) in edges.values():
        T[s_k, f_src[s_e]] = (t_k, f_dst[t_e], s_r != t_r)
        T[t_k, f_src[t_e]] = (s_k, f_dst[s_e], s_r != t_r)

    assert len(T) == 4*6
    return T

In [16]:
class Map2:
    def __init__(self, M, T, O):
        self._M = M
        self._T = T
        self._O = O
        self.x = 0
        self.y = 0
        self.side = 0
        self.facing = Facing.right

    @property
    def size(self):
        return self._M.shape[1]    
    
    def move(self, inst):
        if inst in ['L', 'R']:
            self.facing = self.facing.turn(inst)
        else:
            inst = int(inst)
            self.forward(inst)

    def forward(self, num_steps : int):
        assert num_steps >= 0
        for _ in range(num_steps):
            moved = self.step()
            if not moved:
                return

    def transition(self, dist):
        side, facing, reverse = self._T[self.side, self.facing]
        dist = (self.size -1 - dist) if reverse else dist

        match facing:
            case Facing.up:
                x,y = dist, self.size-1
            case Facing.down:
                x,y = dist, 0
            case Facing.left:
                x,y = self.size-1, dist
            case Facing.right:
                x,y = 0, dist
        
        return facing, side, x ,y

    def step(self):
        match self.facing:
            case Facing.up:
                if self.y == 0:
                    f_n, s_n, x_n, y_n = self.transition(self.x)
                else:
                    f_n, s_n, x_n, y_n = self.facing, self.side, self.x, self.y - 1
            case Facing.down:
                if self.y == self.size - 1:
                    f_n, s_n, x_n, y_n = self.transition(self.x)
                else:
                    f_n, s_n, x_n, y_n = self.facing, self.side, self.x, self.y + 1
            case Facing.left:
                if self.x == 0:
                    f_n, s_n, x_n, y_n = self.transition(self.y)
                else:
                    f_n, s_n, x_n, y_n = self.facing, self.side, self.x - 1, self.y
            case Facing.right:
                if self.x == self.size - 1:
                    f_n, s_n, x_n, y_n = self.transition(self.y)
                else:
                    f_n, s_n, x_n, y_n = self.facing, self.side, self.x + 1, self.y
            
        can_move = (self._M[s_n, y_n, x_n] == '.')
        if can_move:
            self.facing, self.side, self.x, self.y = f_n, s_n, x_n, y_n
        return can_move

    def password(self):
        oy,ox = self._O[self.side]
        return ((self.y+oy+1) * 1000)  + ((self.x+ox+1) * 4) + self.facing.value

In [27]:
def part2(prefix='data'):
    (M, N, O), I = read(prefix, part=2)
    corners = find_corners(N)
    edges = find_edges(corners)
    T = make_transitions(edges)

    print(*edges.items(), sep='\n')
    map = Map2(M,T, O)
    for inst in I:
        map.move(inst)
    return map.password()

In [28]:
part2('test')

(((0, 0, 0), (1, 0, 0)), [(0, 'T', 1), (1, 'T', -1)])
(((1, 0, 0), (0, 0, 0)), [(0, 'T', -1), (1, 'T', 1)])
(((0, 0, 0), (0, 1, 0)), [(0, 'L', 1), (2, 'T', 1)])
(((0, 1, 0), (0, 0, 0)), [(0, 'L', -1), (2, 'T', -1)])
(((0, 1, 0), (1, 1, 0)), [(0, 'B', 1), (3, 'T', 1)])
(((1, 1, 0), (0, 1, 0)), [(0, 'B', -1), (3, 'T', -1)])
(((1, 0, 0), (1, 1, 0)), [(0, 'R', 1), (5, 'R', -1)])
(((1, 1, 0), (1, 0, 0)), [(0, 'R', -1), (5, 'R', 1)])
(((0, 1, 0), (0, 1, 1)), [(3, 'L', 1), (2, 'R', 1)])
(((0, 1, 1), (0, 1, 0)), [(3, 'L', -1), (2, 'R', -1)])
(((0, 1, 1), (1, 1, 1)), [(3, 'B', 1), (4, 'T', 1)])
(((1, 1, 1), (0, 1, 1)), [(3, 'B', -1), (4, 'T', -1)])
(((1, 1, 0), (1, 1, 1)), [(3, 'R', 1), (5, 'T', -1)])
(((1, 1, 1), (1, 1, 0)), [(3, 'R', -1), (5, 'T', 1)])
(((0, 1, 1), (0, 0, 1)), [(4, 'L', 1), (2, 'B', -1)])
(((0, 0, 1), (0, 1, 1)), [(4, 'L', -1), (2, 'B', 1)])
(((0, 0, 1), (1, 0, 1)), [(4, 'B', 1), (1, 'B', -1)])
(((1, 0, 1), (0, 0, 1)), [(4, 'B', -1), (1, 'B', 1)])
(((1, 1, 1), (1, 0, 1)), [(4

5031

In [19]:
part2()

127012